In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Lectura del archivo con la informacion a trabajar optenida de Kaggle
df = pd.read_csv("C:/Users/gerar/Documents/PROYECTO-CD-Premiere-league/data/processed/df_final.csv")
df

In [ ]:
df.dtypes

In [5]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

categorical_cols = ['Venue', 'Result', 'Opponent', 'Team']

label_encoder = LabelEncoder()
for col in categorical_cols:
    df[col] = label_encoder.fit_transform(df[col])

X = df.drop('Result', axis=1)
y = df['Result']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
#0:.Draw, 1:Win, 2:Lose 
df.head()

,Venue,Result,GF,GA,Opponent,xG,xGA,Team
0,0,0,1.0,1.0,6,1.3,1.4,10
1,1,2,3.0,1.0,2,3.0,1.3,10
2,0,2,2.0,1.0,14,0.9,2.0,10
3,1,2,3.0,0.0,1,2.5,0.7,10
4,0,2,3.0,1.0,19,2.5,0.6,10


In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Definir los hiperparámetros que deseas probar
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga']
}

# Crear un modelo de regresión logística
logistic_regression_clf = LogisticRegression(random_state=42)

# Configurar la búsqueda de hiperparámetros usando GridSearchCV
grid_search = GridSearchCV(estimator=logistic_regression_clf, param_grid=param_grid, cv=5)

# Entrenar la búsqueda de hiperparámetros en el conjunto de entrenamiento
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros encontrados
best_params = grid_search.best_params_
print("Mejores parámetros:", best_params)


C:\Users\gerar\Documents\PROYECTO-CD-Premiere-league\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\gerar\Documents\PROYECTO-CD-Premiere-league\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\gerar\Documents\PROYECTO-CD-Premiere-league\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\gerar\Documents\PROYECTO-CD-Premiere-league\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\gerar\Documents\PROYECTO-CD-Premiere-league\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reac

Mejores parámetros: {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}


C:\Users\gerar\Documents\PROYECTO-CD-Premiere-league\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\gerar\Documents\PROYECTO-CD-Premiere-league\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [20]:
 #pip install mlflow
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

name_model = "LogisticRegression"
developer = "Angel De la mora, Regina Flores, Santiago Aguirre"

parameters = {
    "penalty": "l2",
    "C": 1.0,
    "solver": "lbfgs",
    "multi_class": "multinomial", 
    "max_iter": 100,
    "random_state": 42
}

with mlflow.start_run(run_name=name_model):
    
    mlflow.log_param("model", name_model)
    mlflow.log_param("developer", developer)
    
    # Registrar parámetros del modelo
    mlflow.log_params(parameters)
    
    # Entrenar un clasificador de regresión logística
    logistic_regression_clf = LogisticRegression(**parameters)
    logistic_regression_clf.fit(X_train, y_train)
    
    # Realizar predicciones
    predictions = logistic_regression_clf.predict(X_test)
    predictions_proba = logistic_regression_clf.predict_proba(X_test)

    # Calcular métricas
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    roc_auc = roc_auc_score(y_test, predictions_proba, average='weighted', multi_class='ovr')

    # Registrar métricas
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    
    # Registrar el modelo
    mlflow.sklearn.log_model(logistic_regression_clf, "logistic_regression_classifier")


C:\Users\gerar\Documents\PROYECTO-CD-Premiere-league\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\gerar\Documents\PROYECTO-CD-Premiere-league\.venv\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make 

In [2]:
import dagshub
import mlflow

dagshub.init("PROYECTO-CD-Premiere-league", "IlseFlores", mlflow=True)

mlflow.set_tracking_uri('https://dag.com/IlseFlores/PROYECTO-CD-Premiere-league.mlflow')

Initialized MLflow to track repo "IlseFlores/PROYECTO-CD-Premiere-league"

Repository IlseFlores/PROYECTO-CD-Premiere-league initialized!

In [3]:
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

name_model = "LogisticRegression"
developer = "Angel De la mora, Regina Flores, Santiago Aguirre"

parameters = {
    "penalty": "l2",
    "C": 1.0,
    "solver": "lbfgs",
    "multi_class": "multinomial", 
    "max_iter": 100,
    "random_state": 42
}

with mlflow.start_run(run_name=name_model):
    
    mlflow.log_param("model", name_model)
    mlflow.log_param("developer", developer)
    
    # Registrar parámetros del modelo
    mlflow.log_params(parameters)
    
    # Entrenar un clasificador de regresión logística
    logistic_regression_clf = LogisticRegression(**parameters)
    logistic_regression_clf.fit(X_train, y_train)
    
    # Realizar predicciones
    predictions = logistic_regression_clf.predict(X_test)
    predictions_proba = logistic_regression_clf.predict_proba(X_test)

    # Calcular métricas
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    roc_auc = roc_auc_score(y_test, predictions_proba, average='weighted', multi_class='ovr')

    # Registrar métricas
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    
    # Registrar el modelo
    mlflow.sklearn.log_model(logistic_regression_clf, "logistic_regression_classifier")


MlflowException: API request to https://dag.com/IlseFlores/PROYECTO-CD-Premiere-league.mlflow/api/2.0/mlflow/runs/create failed with exception HTTPSConnectionPool(host='dag.com', port=443): Max retries exceeded with url: /IlseFlores/PROYECTO-CD-Premiere-league.mlflow/api/2.0/mlflow/runs/create (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001BFD7D1CF50>: Failed to resolve 'dag.com' ([Errno 11001] getaddrinfo failed)"))

In [ ]:

import dagshub
import mlflow

dagshub.init("PROYECTO-CD-Premiere-league", "IlseFlores", mlflow=True)

mlflow.set_tracking_uri('https://dagshub.com/IlseFlores/PROYECTO-CD-Premiere-league.mlflow')